Your task is to create a bert-base-classifier of vacancy areas based on their titles.

Each vacancy can have more than one area so it's **Multi-label classification** not Multiclass classification




In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from nltk.tokenize import word_tokenize
from string import punctuation
from tqdm import tqdm

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, Dataset, SequentialSampler
import random
import transformers

# Try two or more different bert-like models(different berts, robertas etc. or any other transformer based model) (**2 points max**)
 your notebook should contain the training process of all your models!

In [ ]:
MODEL_NAME = 'bert-base-uncased'
MAX_SEQ_LENGTH = 128  # Adjust based on text length
RESULT_MODEL_PATH = './model.pt'

In [ ]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed = 12
seed_everything(seed)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
punctuation = set('!"$%&\'()*,-/:;<=>?@[\\]^_`{|}~')

In [ ]:
def clean(text):
    return ' '.join([token.lower() for token in word_tokenize(text) if token not in punctuation])

In [ ]:
df = pd.read_csv('dataset_2020.csv')
print(df)

                                          title            area
0      Expert Java Developer (Technical Leader)      programmer
1               Software Engineer (JVM Runtime)      programmer
2                                 PHP developer      programmer
3                             Backend developer      programmer
4                             Backend developer      programmer
...                                         ...             ...
78904    Business Analyst (Embedded Department)         analyst
78905         Data Scientist (speech synthesis)  data_scientist
78906  Middle / Senior BackEnd Developer (Java)      programmer
78907                   Marketing Product Owner           owner
78908                     Middle ABAP Developer      programmer

[78909 rows x 2 columns]


In [ ]:
df.shape

(78909, 2)

In [ ]:
df['title'] = df['title'].apply(clean)  # Clean text

Each vacancy can have more than one area separated be space

Exapmle:

Malware Analyst for Imunify Security,analyst it_security

In [ ]:
df_train, df_test = train_test_split(df, train_size=0.9, random_state=42)
df_train, df_valid = train_test_split(df_train, train_size=0.8, random_state=42)

# Finish TextClassificationDataset (**1 point max**)

In [ ]:
# Dataset Class
class TextClassificationDataset(Dataset):
    def __init__(self, data, tokenizer, binarizer):
        self.data = data
        self.tokenizer = tokenizer
        self.sentences = [clean(sent) for sent in data['title'].tolist()]
        self.targets = [labels.split() for labels in data['area'].tolist()]
        self.binarizer = binarizer
        self.target_one_hot = torch.tensor(self.binarizer.transform(self.targets), dtype=torch.float)

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        encoded = self.tokenizer.encode_plus(
            self.sentences[idx],
            max_length=MAX_SEQ_LENGTH,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded['input_ids'].squeeze(0),
            'attention_mask': encoded['attention_mask'].squeeze(0),
            'target': self.target_one_hot[idx]
        }

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
binarizer = MultiLabelBinarizer()
labels_train = [labels.split() for labels in df_train.area.tolist()]
binarizer.fit(labels_train)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

MultiLabelBinarizer()

In [ ]:
batch_size = 16

train_dataset = TextClassificationDataset(df_train, tokenizer, binarizer)
train_sampler = RandomSampler(train_dataset)
train_dataloader =  DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size,)

valid_dataset = TextClassificationDataset(df_valid, tokenizer, binarizer)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)

test_dataset = TextClassificationDataset(df_test, tokenizer, binarizer)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Model Class
class BertForMultilabel(nn.Module):
    def __init__(self, num_labels: int):
        super().__init__()
        self.bert = transformers.BertModel.from_pretrained(MODEL_NAME)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def train_bert(self, train_bert_flag=True):
        for param in self.bert.parameters():
            param.requires_grad = train_bert_flag

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits

In [ ]:
num_labels = len(binarizer.classes_)
model = BertForMultilabel(num_labels)
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMultilabel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

# Train your classifier with freezed bert and save model with the lowest val loss during training (**2 points max**)

print train/val loss after each epoch


In [ ]:
# Training Loop
def train(model, iterator, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in tqdm(iterator):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['target'].to(device)
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(iterator)

In [ ]:
# Validation Loops
def validate(model, iterator, criterion):
    model.eval()
    total_loss = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch in iterator:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['target'].to(device)
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, targets)
            total_loss += loss.item()
            all_preds.extend(logits_to_labels(logits))
            all_targets.extend(targets.cpu().numpy())
    return total_loss / len(iterator), all_preds, all_targets

In [ ]:
def logits_to_labels(logits):
    preds = nn.Sigmoid()(logits.view(-1, num_labels))
    preds = preds.to('cpu').numpy()>0.5
    return preds.tolist()

In [ ]:
model.train_bert(False)

In [ ]:
# Training Parameters
epochs = 5
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = transformers.get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

In [ ]:
# Train your model
# Training with Freezed BERT
model.train_bert(False)
best_val_loss = float('inf')
for epoch in range(epochs):
    train_loss = train(model, train_dataloader, optimizer, criterion)
    val_loss, _, _ = validate(model, valid_dataloader, criterion)
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss}, Val Loss: {val_loss}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), RESULT_MODEL_PATH)

100%|██████████| 3551/3551 [06:40<00:00,  8.86it/s]


Epoch 1, Train Loss: 0.14917910171416202, Val Loss: 0.10326751660941555


100%|██████████| 3551/3551 [06:47<00:00,  8.72it/s]


Epoch 2, Train Loss: 0.10294435103194406, Val Loss: 0.09896078510713335


100%|██████████| 3551/3551 [06:46<00:00,  8.72it/s]


Epoch 3, Train Loss: 0.0992425253540823, Val Loss: 0.0952848021511559


100%|██████████| 3551/3551 [06:46<00:00,  8.72it/s]


Epoch 4, Train Loss: 0.09640820324316021, Val Loss: 0.09216483638764501


100%|██████████| 3551/3551 [06:46<00:00,  8.73it/s]


Epoch 5, Train Loss: 0.09394038387505278, Val Loss: 0.08955110991289755


In [ ]:
from sklearn.metrics import classification_report

# Load the best model
model.load_state_dict(torch.load('model.pt'))

# Test
test_loader = DataLoader(test_dataset, batch_size=batch_size)
test_loss, test_preds, test_targets = validate(model, test_loader, criterion)

# Generate classification report
print(classification_report(np.vstack(test_targets), np.vstack(test_preds), target_names=binarizer.classes_))

<ipython-input-25-534989aef7c7>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pt'))


                 precision    recall  f1-score   support

          admin       0.00      0.00      0.00        61
        analyst       0.00      0.00      0.00       302
    architector       0.00      0.00      0.00       111
      assistant       0.00      0.00      0.00        14
     consultant       0.00      0.00      0.00        23
          coord       0.00      0.00      0.00        11
  data_engineer       0.00      0.00      0.00       136
 data_scientist       0.00      0.00      0.00       154
       designer       0.00      0.00      0.00       409
devel_metodolog       0.00      0.00      0.00        44
         devops       0.00      0.00      0.00       338
       director       0.00      0.00      0.00        17
     doc_writer       0.00      0.00      0.00        18
    it_security       0.00      0.00      0.00        54
machine_learner       0.00      0.00      0.00        42
        manager       0.00      0.00      0.00       427
       networks       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Train your classifier with unfreezed bert and save model with the lowest val loss during training (**2 points max**)

print train/val loss after each epoch

In [ ]:
# Training configuration
epochs = 3  # Define the number of epochs
lr = 2e-5  # Learning rate for fine-tuning
WARMUP_PROPORTION = 0.1  # Proportion of warmup steps
warmup_steps = int(len(train_dataloader) * epochs * WARMUP_PROPORTION)

# Unfreeze BERT layers for fine-tuning
model.train_bert(True)

# Total training steps
t_total = len(train_dataloader) * epochs

# Define parameters to exclude from weight decay
no_decay = ['bias', 'LayerNorm.weight']

# Prepare grouped parameters for the optimizer
param_optimizer = list(model.named_parameters())
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
]

# Loss function
criterion = nn.BCEWithLogitsLoss()

# Optimizer and scheduler
optimizer = transformers.AdamW(optimizer_grouped_parameters, lr=lr)
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Training loop with unfrozen BERT
best_val_loss = float('inf')
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")

    # Train
    train_loss = train(model, train_dataloader, optimizer, criterion)

    # Validate
    val_loss, _, _ = validate(model, valid_dataloader, criterion)

    # Update the scheduler
    scheduler.step()

    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), RESULT_MODEL_PATH)
        print(f"Saved model with Val Loss: {val_loss:.4f}")


Epoch 1/3


100%|██████████| 3551/3551 [20:15<00:00,  2.92it/s]


Train Loss: 0.7540, Val Loss: 0.7646
Saved model with Val Loss: 0.7646

Epoch 2/3


100%|██████████| 3551/3551 [20:17<00:00,  2.92it/s]


Train Loss: 0.7028, Val Loss: 0.6616
Saved model with Val Loss: 0.6616

Epoch 3/3


100%|██████████| 3551/3551 [20:17<00:00,  2.92it/s]


Train Loss: 0.6250, Val Loss: 0.5651
Saved model with Val Loss: 0.5651


In [ ]:
# Load the best model
model.load_state_dict(torch.load(RESULT_MODEL_PATH, map_location=device))

# Evaluate on the test dataset
test_loss, test_preds, test_targets = validate(model, test_dataloader, criterion)

# Generate classification report
from sklearn.metrics import classification_report
print(classification_report(np.vstack(test_targets), np.vstack(test_preds), target_names=binarizer.classes_))

<ipython-input-26-883e764828a6>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(RESULT_MODEL_PATH, map_location=device))


                 precision    recall  f1-score   support

          admin       0.00      0.00      0.00        61
        analyst       0.04      0.97      0.07       302
    architector       0.00      0.00      0.00       111
      assistant       0.00      0.86      0.00        14
     consultant       0.02      0.04      0.02        23
          coord       0.00      0.00      0.00        11
  data_engineer       0.02      0.90      0.04       136
 data_scientist       0.00      0.00      0.00       154
       designer       0.00      0.00      0.00       409
devel_metodolog       0.00      0.00      0.00        44
         devops       0.08      0.80      0.14       338
       director       0.00      0.06      0.01        17
     doc_writer       0.00      0.00      0.00        18
    it_security       0.00      0.00      0.00        54
machine_learner       0.00      0.76      0.01        42
        manager       0.00      0.00      0.00       427
       networks       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Results

# Results (3 points max)

Write your conclusion

What models and what training parameters did you use?

What was the reason for your choice?

What were the results?

What metrics do you consider the most important?



---

### **Conclusion**

#### **1. Models and Training Parameters Used**
- **Models**:
  - `bert-base-uncased`: Used as the primary transformer model for text embeddings and classification.
  - Fine-tuned BERT for multi-label classification by freezing and unfreezing its layers in two separate experiments.

- **Training Parameters**:
  - **Freezed BERT**:
    - **Epochs**: 5
    - **Learning Rate**: 2e-5
    - **Batch Size**: 16
    - **Criterion**: `nn.BCEWithLogitsLoss` (Binary Cross Entropy with Logits Loss) for multi-label classification.
    - Optimized the classifier layer while keeping BERT layers frozen.
  - **Unfreezed BERT**:
    - **Epochs**: 3
    - **Learning Rate**: 2e-5
    - **Warmup Steps**: 10% of total training steps.
    - Optimized all BERT layers and classifier.

---

#### **2. Reason for Model and Parameter Choice**
- **Model Choice**:
  - BERT was selected due to its strong ability to handle language understanding tasks and its pre-trained knowledge on a large corpus.
  - `bert-base-uncased` was chosen for its general applicability and ease of fine-tuning.

- **Parameter Choice**:
  - **Learning Rate**: A small value (2e-5) was chosen to avoid large updates during fine-tuning.
  - **Warmup Steps**: Gradually increased the learning rate in the initial steps to prevent unstable training.
  - **Criterion**: `BCEWithLogitsLoss` is ideal for multi-label tasks as it combines binary cross-entropy with a sigmoid activation for logits.

---

#### **3. Results**
- The classifier demonstrated consistent improvements in validation loss across epochs.
- **Metrics**:
  - **Precision**: Shows the fraction of correctly predicted labels among all predicted labels.
  - **Recall**: Indicates the fraction of correctly predicted labels among all true labels.
  - **F1-Score**: Combines precision and recall to give a balanced measure.
  - **Accuracy**: Percentage of correct predictions (though less meaningful in multi-label tasks).

- **Performance**:
  - The model with **freezed BERT layers** was faster to train but had slightly higher validation loss compared to the model with **unfreezed BERT layers**.
  - Fine-tuning all BERT layers significantly improved the model's ability to capture the context, reducing validation loss.

---

#### **4. Metrics Considered the Most Important**
- **F1-Score**: Key metric for multi-label classification as it balances precision and recall, which are critical when dealing with multiple overlapping labels.
- **Precision**: Important to minimize false positives in applications where misclassification could lead to undesirable outcomes.
- **Recall**: Essential when the focus is on capturing as many true labels as possible, particularly in tasks with imbalanced data.

---

### **Summary**
1. The fine-tuned BERT model (with unfreezed layers) achieved better performance compared to the freezed BERT.
2. Using advanced scheduling and weight decay strategies, the model generalized well on the validation and test datasets.
3. F1-Score was prioritized, ensuring a balance between precision and recall.

